In [1]:
import duckdb

In [4]:
duckdb.sql("INSTALL httpfs; LOAD httpfs;")
duckdb.sql("INSTALL iceberg; LOAD iceberg;")

In [3]:
duckdb.sql("SET s3_endpoint='127.0.0.1:9000';")
duckdb.sql("SET s3_access_key_id='minio';")
duckdb.sql("SET s3_secret_access_key='minio123';")
duckdb.sql("SET s3_region='us-east-1';")
duckdb.sql("SET s3_url_style='path';")
duckdb.sql("SET s3_use_ssl=false;")

In [7]:
result = duckdb.sql("""
SELECT * FROM iceberg_scan('s3://warehouse/goodwiki/ingestion_etl')
""").df()

In [8]:
result

,pageid,title,revid,description,categories,markdown
0,57185536,Georgia Hopley,1163683705,American journalist and temperance advocate,"[1858 births, 1944 deaths, 19th-century Americ...",Georgianna Eliza Hopley (1858–1944) was an Ame...
1,15394015,Willis Ward,1170257280,Track and field athlete and American football ...,"[1912 births, 1983 deaths, 20th-century Africa...","Willis Franklin Ward (December 28, 1912 – Dece..."
2,62958021,Instant Replay Game,1166130448,Notable American football game,"[1989 National Football League season, 1989 in...","The Instant Replay Game, also known as the Ast..."
3,17546,Louvre,1172724876,"Art museum in Paris, France","[1793 establishments in France, Archaeological...","The Louvre (English: /ˈluːv(rə)/ ), or the Lou..."
4,13280513,Launch Party,1150875056,None,"[2007 American television episodes, The Office...","""Launch Party"" is the fifth and sixth episode ..."
...,...,...,...,...,...,...
44749,65986818,Let Us Continue,1148889149,1963 speech by U.S. President Lyndon B. Johnson,"[1963 in American politics, 1963 in Washington...",Let Us Continue was a speech that 36th Preside...
44750,23240578,Odin-class coastal defense ship,1161437920,Coastal defense ship class of the German Imper...,"[Coastal defense ship classes, Odin-class coas...",The Odin class was a pair of coastal defense s...
44751,209236,Pest (organism),1173463954,Organism harmful to humans/our concerns,"[Insects in culture, Pests (organism)]",A pest is any organism harmful to humans or hu...
44752,62440986,44 Union Square,1170828953,"Office building in Manhattan, New York","[1929 establishments in New York City, New Yor...","44 Union Square, also known as 100 East 17th S..."


In [ ]:
#generate mschema


In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
model_path = "XGenerationLab/XiYanSQL-QwenCoder-7B-2504"
llm = LLM(model=model_path, tensor_parallel_size=8)
tokenizer = AutoTokenizer.from_pretrained(model_path)
sampling_params = SamplingParams(
    n=1,
    temperature=0.1,
    max_tokens=1024
)

## dialects -> ['SQLite', 'PostgreSQL', 'MySQL']
prompt = nl2sqlite_template_cn.format(dialect="", db_schema="", question="", evidence="")
message = [{'role': 'user', 'content': prompt}]
text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)
outputs = llm.generate([text], sampling_params=sampling_params)
response = outputs[0].outputs[0].text
